[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/CO-CONNECT/co-connect-tools/HEAD)


## Installing

The best way is to install the module via `pip`. 

In [ ]:
!pip install co-connect-tools -q

## Start the Tool

To start the ETLTool we can import it from the `coconnect` module we just installed

In [1]:
import coconnect
etltool = coconnect.ETLTool()

2021-02-19 11:14:07 - ETLTool - INFO - Starting the tool


## Load Inputs

To run the tool you need to load some input datasets, and specify how to map the fields 

The data will be loaded into pandas dataframes that we'll use for some visualisations of what the input `csv` files will look like

### Source data

This data is synthetic data that has been produced by [OHDSI](http://ohdsi.org/) which simply details a record of patients.

_Note: that these example data files will be stored in `<install_dir>/lib/python3.8/site-packages/coconnect/`, a directory that `ETLTool` will be looking in. For your own files, you should specify the full path to the inputs_

In [2]:
f_input_data = 'sample_input_data/patients_sample.csv'
etltool.load_input_data([f_input_data])

2021-02-19 11:14:07 - ETLTool - INFO - found the following input tables: ['patients_sample.csv']


Verify what files have been loaded, by default the input dataset is mapped to to a name via `/path/<name>.csv`

In [3]:
etltool.get_input_names()

['patients_sample.csv']

Sample (3 entries) what this input data looks like. __Note__ becareful using this method with a large dataset

In [5]:
df_input = etltool.get_input_df('patients_sample.csv')
df_input.sample(3)

,ID,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,ZIP
5,48875d6f-3aca-4192-a602-1c5ec8e151d8,1984-04-03,NaN,999-82-5042,S99959890,X72913497X,Mr.,Shon148,Zulauf375,NaN,NaN,M,white,italian,M,Tewksbury,1043 Torp Orchard Unit 8,Foxborough,Massachusetts,2035
0,f4ce17a0-720e-46e8-8a12-befc90fab3bc,2002-08-11,NaN,999-61-4021,S99985608,NaN,NaN,Lucila204,Langosh790,NaN,NaN,NaN,white,english,F,Braintree Town,943 Barton Trailer,North Adams,Massachusetts,1247
12,d948ceae-4d3d-49e7-9bc4-d6cb0c944017,2005-03-19,NaN,999-58-7722,NaN,NaN,NaN,Nguyet780,Gerlach374,NaN,NaN,NaN,white,italian,F,Wilmington,1063 Russel Lodge Apt 82,Stoughton,Massachusetts,2072


### Structural Mapping

Next we use another `csv` file to define how to map different fields in the source data to a [Common Data Model (CDM)](https://www.ohdsi.org/data-standardization/the-common-data-model/).

In this example, the CDM that the source data (`patients_sample`)  is being mappped to is the `person` CDM.

There are three rules defined:
1. Performs a straight one-to-one mapping between the field `id` in the source field to the `person_id` field of the `person` CDM
2. Performs a mapping with the operation/function `extract year` 
3. Performs a term mapping which is defined in the term mapping `csv` file, see the next section for more information 


In [6]:
f_structural_mapping = 'sample_input_data/rules1.csv'
etltool.load_structural_mapping(f_structural_mapping) 
etltool.get_structural_mapping_df()

2021-02-19 11:15:23 - ETLTool - INFO - Loaded the structural mapping with 3 rules


,rule_id,destination_table,destination_field,source_table,source_field,term_mapping,coding_system,operation
0,0,person,person_id,patients_sample.csv,id,n,user defined,n
1,1,person,year_of_birth,patients_sample.csv,birthdate,n,user defined,EXTRACT_YEAR
2,2,person,gender_concept_id,patients_sample.csv,gender,y,user defined,n


#### Testing operations
The 2nd rule defined the operation `extract year`, this is a default operation defined in `etltool`, here is a quick example of how it works..

Load the function

In [11]:
fn_extract_year = etltool.allowed_operations['EXTRACT_YEAR']
fn_extract_year

<bound method ETLTool.get_year_from_date of <coconnect.etltool.ETLTool object at 0x10a99c070>>

For example, taking the `BIRTHDATE` columns, which looks like:

In [12]:
df_input['BIRTHDATE'].head(4)

0    2002-08-11
1    2002-10-25
2    1990-02-24
3    1966-02-11
Name: BIRTHDATE, dtype: object

The function can be used to easily extract the year from the date

In [13]:
fn_extract_year(df_input.head(4),column='BIRTHDATE')

0    2002
1    2002
2    1990
3    1966
Name: BIRTHDATE, dtype: int64

### Term Mapping

In the term mapping, the structural mapping `rule_id` is mapped telling us how to map a source term to a destination term, i.e. if the source term is `M` then the output should be `8507`

In [14]:
f_term_mapping = 'sample_input_data/rules2.csv'
etltool.load_term_mapping(f_term_mapping)
etltool.get_term_mapping_df()

2021-02-19 11:15:53 - ETLTool - INFO - Loaded the term mapping with 2 rules


,source_term,destination_term
rule_id,,
2,M,8507
2,F,8532


### Run the tool

In [15]:
etltool.run()

2021-02-19 11:15:54 - ETLTool - INFO - starting to run
2021-02-19 11:15:54 - ETLTool - INFO - Destination tables to create... ['person']
2021-02-19 11:15:54 - ETLTool - INFO - Done with tool initialisation...
2021-02-19 11:15:54 - ETLTool - INFO - Starting ETL to CDM
2021-02-19 11:15:54 - ETLTool - INFO - Now running on Table "person"
2021-02-19 11:15:54 - ETLTool - INFO - Loaded the CDM for person
2021-02-19 11:15:54 - ETLTool - INFO - The CDM for "person" has 18, you have mapped 3 leaving 15 fields unmapped
2021-02-19 11:15:54 - ETLTool - INFO - Processing 0 with length 19
2021-02-19 11:15:54 - ETLTool - INFO - Working on person_id
2021-02-19 11:15:54 - ETLTool - INFO - Working on year_of_birth
2021-02-19 11:15:54 - ETLTool - INFO - Working on gender_concept_id
2021-02-19 11:15:54 - ETLTool - INFO - chunk[0] completed: Final dataframe with 19 rows and 3 columns created
2021-02-19 11:15:54 - ETLTool - INFO - Creating a new folder: ./data//person/
2021-02-19 11:15:54 - ETLTool - INFO -

We can finally get the output in a dataframe

In [17]:
etltool.get_output_df('person')

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,f4ce17a0-720e-46e8-8a12-befc90fab3bc,8532,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b4339f80-9313-4437-8664-cffdca3c5e9a,8532,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,edd5a63e-f86a-4fb9-b9ff-b26c5506c96b,8507,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,525fdbdc-959e-472f-8986-d1a492c89d45,8532,1966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57143e43-c999-4c34-924e-85366d2cac70,8532,1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,48875d6f-3aca-4192-a602-1c5ec8e151d8,8507,1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,cd2ceea4-60ab-4d06-a242-75ca09565cfd,8532,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ab88704a-530f-4f71-b9c3-43680747b607,8507,1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0799fb7f-1bca-475b-b4b5-e2bdb12cc2ea,8507,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,d41715db-9a90-495a-9f91-aaa16ad9442e,8507,1969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
